# DistilBERT model
This notebook trains the DistilBERT model and exports a set of predictions for a test dataset.


**Trained on**: SemEval (2019)
**Tested on**: SemEval (2019)

First we need to install the required packages.

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.1 MB/s 
     |████████████████████████████████| 596 kB 70.7 MB/s 
     |████████████████████████████████| 6.6 MB 66.3 MB/s 
     |████████████████████████████████| 101 kB 11.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 4.2 MB/s 
     |████████████████████████████████| 212 kB 71.9 MB/s 
     |████████████████████████████████| 140 kB 74.6 MB/s 
     |████████████████████████████████| 1.1 MB 69.8 MB/s 
     |████████████████████████████████| 127 kB 92.1 MB/s 
     |████████████████████████████████| 94 kB 2.5 MB/s 
     |████████████████████████████████| 271 kB 91.3 MB/s 
     |████████████████████████████████| 144 kB 64.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
!pip install tweet-preprocessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from google.colab import drive

# Mount drive for loading the data
drive.mount('/content/drive')

import sys
sys.path.append("drive/MyDrive/Colab Notebooks")

from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, DistilBertTokenizer
import numpy as np
from datasets import load_metric, load_dataset, Dataset
from sklearn.model_selection import train_test_split
import pickle
import torch
from google.colab import drive
import preprocessor as p
import html
from torch.utils.data import DataLoader
import torch.nn.functional as nn
from reader import Reader

FILENAME = "drive/MyDrive/Colab Notebooks/data/sem_eval_all.pkl"

Mounted at /content/drive


## Split and tokenize the datasets

In [6]:
class HateDataset(torch.utils.data.Dataset):
    """Dataset class used for combining the data encodings and labels."""
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def preprocess(data):
    """Preprocesses the data by tokenizing URLs and MENTIONS since these do not
    contain any valuable information."""
    p.set_options(p.OPT.URL, p.OPT.MENTION)

    return list(map(lambda text: p.tokenize(html.unescape(text)), data))

In [7]:
reader = Reader(filename=FILENAME)
X, y = reader.load()
X = preprocess(X)

mapping = {'hate': 1, 'none': 0}
y = [mapping[b] for b in y]

# Split dataset into train, test, and validation
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, stratify=y, test_size=0.10)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=10, test_size=.2)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize all datasets
train_encodings = tokenizer(X_train, truncation=True, padding=True)
val_encodings = tokenizer(X_val, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

# Combine the encodings with the labels to Torch datasets
train_dataset = HateDataset(train_encodings, y_train)
val_dataset = HateDataset(val_encodings, y_val)
test_dataset = HateDataset(test_encodings, y_test)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

## Load existing model

In [8]:
model = torch.load(F"drive/MyDrive/Colab Notebooks/output/distilbert-semeval-calibrated.pth")
model.eval()

ModelWithTemperature(
  (model): DistilBertForSequenceClassification(
    (distilbert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (

## Run model on the test dataset
The predicted output logits of the test dataset are converted to class probabilities using Softmax.

In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

logits = []
labels = []

test_loader = DataLoader(test_dataset)

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        label = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits.append(outputs.tolist()[0])
        labels.append(label)

probabilities = nn.softmax(torch.tensor(logits), dim=1)
print(probabilities.tolist()[:10])

[[0.9735488295555115, 0.02645118348300457], [0.8830448389053345, 0.11695511639118195], [0.9253570437431335, 0.07464297860860825], [0.30723127722740173, 0.6927687525749207], [0.7239320874214172, 0.27606794238090515], [0.7789956331253052, 0.2210044413805008], [0.9693713188171387, 0.03062872402369976], [0.6045182943344116, 0.395481675863266], [0.9589912295341492, 0.04100877791643143], [0.11887717247009277, 0.8811227679252625]]


In [10]:
predictions = torch.argmax(probabilities, dim=-1)
metric = load_metric("accuracy")
accuracy = metric.compute(predictions=predictions, references=labels)
print("Test dataset: ", accuracy)

Test dataset:  {'accuracy': 0.7683333333333333}


## Export predictions

In [11]:
import numpy as np

def to_predictions(predictions, labels):
    predicted_classes = np.argmax(predictions, 1)
    return list(map(lambda predicted_values, predicted_class, actual_class:
                    {'predicted_class': class_name(predicted_class),
                     'actual_class': class_name(actual_class),
                     'predicted_value': predicted_values[predicted_class].item(),
                     'text': None},
                    predictions, predicted_classes, labels))
def class_name(index):
    if index == 0:
        return "None"
    elif index == 1:
        return "Hate"

predictions_info = to_predictions(probabilities, labels)
pickle.dump(predictions_info, open("drive/MyDrive/Colab Notebooks/output/distilbert-semeval-semeval.p", "wb"))